#Change Data Feed
- delta log tracks dataset level changes like hey its the file for the new data, or this is the file which is no longer valid
- we have deletion vector which tracks row level changes, but that is for internal use of databricks 
- what we have for us is CDF(change data feed) 
- This allows us to track Row level changes
- This makes very easy to track and query for the data which is inserted, deleted or updated

**Turning CDF for all new tables**

In [0]:
%sql
SET spark.databricks.delta.properties.defaults.enableChangeDataFeed=true;

key,value
spark.databricks.delta.properties.defaults.enableChangeDataFeed,true


In [0]:
%sql
DROP TABLE IF EXISTS demo;
CREATE TABLE demo (id INT ,name STRING);

INSERT INTO demo VALUES (1,"A"),(2,"B"),(3,"C")

num_affected_rows,num_inserted_rows
3,3


In [0]:
%fs ls dbfs:/user/hive/warehouse/demo

path,name,size,modificationTime
dbfs:/user/hive/warehouse/demo/_delta_log/,_delta_log/,0,0
dbfs:/user/hive/warehouse/demo/part-00000-09dbc0dc-c60f-4455-ada0-4ba10bc94fdd-c000.snappy.parquet,part-00000-09dbc0dc-c60f-4455-ada0-4ba10bc94fdd-c000.snappy.parquet,694,1738573089000


Turning CSD for this table and performing some changes

In [0]:
%sql
ALTER TABLE demo SET TBLPROPERTIES (delta.enableChangeDataFeed= true)

In [0]:
%sql

INSERT INTO demo VALUES (4,"D"),(5,"E");

num_affected_rows,num_inserted_rows
2,2


In [0]:
%sql
UPDATE demo 
SET name = "Tiger"
where id=3;

DELETE FROM demo
WHERE id=2;

num_affected_rows
1


In [0]:
%fs ls dbfs:/user/hive/warehouse/demo

path,name,size,modificationTime
dbfs:/user/hive/warehouse/demo/_change_data/,_change_data/,0,0
dbfs:/user/hive/warehouse/demo/_delta_log/,_delta_log/,0,0
dbfs:/user/hive/warehouse/demo/part-00000-09dbc0dc-c60f-4455-ada0-4ba10bc94fdd-c000.snappy.parquet,part-00000-09dbc0dc-c60f-4455-ada0-4ba10bc94fdd-c000.snappy.parquet,694,1738573089000
dbfs:/user/hive/warehouse/demo/part-00000-2abdba6b-f208-40be-8378-8e689025c0cb.c000.snappy.parquet,part-00000-2abdba6b-f208-40be-8378-8e689025c0cb.c000.snappy.parquet,914,1738573099000
dbfs:/user/hive/warehouse/demo/part-00000-600e089d-ede6-4488-abae-f9f4d043c959-c000.snappy.parquet,part-00000-600e089d-ede6-4488-abae-f9f4d043c959-c000.snappy.parquet,685,1738573094000
dbfs:/user/hive/warehouse/demo/part-00000-9215102d-7ac8-439a-a240-317be1dabc18.c000.snappy.parquet,part-00000-9215102d-7ac8-439a-a240-317be1dabc18.c000.snappy.parquet,923,1738573097000


In [0]:
%sql
DESC HISTORY demo;

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
5,2025-02-03T08:58:19Z,5755764547042441,inaya998877@gmail.com,DELETE,"Map(predicate -> [""(id#218542 = 2)""])",null,List(4239566892388268),0203-052121-nggyovx3,4,WriteSerializable,false,"Map(numRemovedFiles -> 1, numRemovedBytes -> 923, numCopiedRows -> 2, numDeletionVectorsAdded -> 0, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 1, executionTimeMs -> 1094, numDeletionVectorsUpdated -> 0, numDeletedRows -> 1, scanTimeMs -> 508, numAddedFiles -> 1, numAddedBytes -> 914, rewriteTimeMs -> 586)",null,Databricks-Runtime/15.4.x-scala2.12
4,2025-02-03T08:58:17Z,5755764547042441,inaya998877@gmail.com,UPDATE,"Map(predicate -> [""(id#217912 = 3)""])",null,List(4239566892388268),0203-052121-nggyovx3,3,WriteSerializable,false,"Map(numRemovedFiles -> 1, numRemovedBytes -> 694, numCopiedRows -> 2, numDeletionVectorsAdded -> 0, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 1, executionTimeMs -> 1490, numDeletionVectorsUpdated -> 0, scanTimeMs -> 752, numAddedFiles -> 1, numUpdatedRows -> 1, numAddedBytes -> 923, rewriteTimeMs -> 738)",null,Databricks-Runtime/15.4.x-scala2.12
3,2025-02-03T08:58:14Z,5755764547042441,inaya998877@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(4239566892388268),0203-052121-nggyovx3,2,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 2, numOutputBytes -> 685)",null,Databricks-Runtime/15.4.x-scala2.12
2,2025-02-03T08:58:12Z,5755764547042441,inaya998877@gmail.com,SET TBLPROPERTIES,"Map(properties -> {""delta.enableChangeDataFeed"":""true""})",null,List(4239566892388268),0203-052121-nggyovx3,1,WriteSerializable,true,Map(),null,Databricks-Runtime/15.4.x-scala2.12
1,2025-02-03T08:58:09Z,5755764547042441,inaya998877@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(4239566892388268),0203-052121-nggyovx3,0,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 3, numOutputBytes -> 694)",null,Databricks-Runtime/15.4.x-scala2.12
0,2025-02-03T08:58:07Z,5755764547042441,inaya998877@gmail.com,CREATE TABLE,"Map(partitionBy -> [], clusterBy -> [], description -> null, isManaged -> true, properties -> {""delta.enableChangeDataFeed"":""true""}, statsOnLoad -> false)",null,List(4239566892388268),0203-052121-nggyovx3,null,WriteSerializable,true,Map(),null,Databricks-Runtime/15.4.x-scala2.12


In [0]:
%sql
select * from demo

id,name
1,A
3,Tiger
4,D
5,E


In [0]:
%sql
SELECT * FROM table_changes('demo',2 )

id,name,_change_type,_commit_version,_commit_timestamp
3,C,update_preimage,4,2025-02-03T08:58:17Z
3,Tiger,update_postimage,4,2025-02-03T08:58:17Z
2,B,delete,5,2025-02-03T08:58:19Z
4,D,insert,3,2025-02-03T08:58:14Z
5,E,insert,3,2025-02-03T08:58:14Z


**Processing deleted**
- **Use Case**: we need to send a deleted data to another team, into the table _demo_deleted_data_ in "real time"

In [0]:
%python 
dbutils.fs.rm("dbfs:/user/hive/warehouse/demo_delted_data",True)
# this is done if the directory is not empty, so to make it empty

True

In [0]:
%sql
DROP TABLE IF EXISTS demo_deleted_data;

CREATE TABLE demo_deleted_data (id INT, name STRING);

In [0]:
%python

dbutils.fs.rm("/FileStore/checkpointDirCD", recurse=True)
spark.readStream.format("delta")\
    .option("readChangeFeed","true")\
    .option("startingVersion",2)\
    .table("demo")\
    .filter("_change_type='delete' ")\
    .select("id","name")\
    .writeStream\
        .outputMode("append")\
        .option("checkpointLocation","/FileStore/checkpointDirCD")\
        .table("demo_delted_data")


this will streamline the process as any data gets deleted will be stored in this table at any time

In [0]:
%sql
SELECT * FROM demo_deleted_data

id,name


In [0]:
%sql
DELETE FROM demo
WHERE id=3

num_affected_rows
1


this deleted data will go in demo_deleted_data

In [0]:
%sql
SELECT * FROM table_changes('demo',2 );

id,name,_change_type,_commit_version,_commit_timestamp
3,C,update_preimage,4,2025-02-03T08:58:17Z
3,Tiger,update_postimage,4,2025-02-03T08:58:17Z
3,Tiger,delete,6,2025-02-03T08:58:30Z
2,B,delete,5,2025-02-03T08:58:19Z
4,D,insert,3,2025-02-03T08:58:14Z
5,E,insert,3,2025-02-03T08:58:14Z


In [0]:
%sql
SELECT * FROM demo_deleted_data

id,name


What else

In [0]:
%sql
SELECT * FROM table_changes('demo','Timestamp')
-- fill the timestamps accordingly

com.databricks.sql.transaction.tahoe.DeltaAnalysisException: [DELTA_TIMESTAMP_INVALID] The provided timestamp ('Timestamp') cannot be converted to a valid timestamp.
	at com.databricks.sql.transaction.tahoe.DeltaErrorsBase.timestampInvalid(DeltaErrors.scala:1470)
	at com.databricks.sql.transaction.tahoe.DeltaErrorsBase.timestampInvalid$(DeltaErrors.scala:1469)
	at com.databricks.sql.transaction.tahoe.DeltaErrors$.timestampInvalid(DeltaErrors.scala:3573)
	at com.databricks.sql.transaction.tahoe.DeltaTimeTravelSpec.getTimestamp(DeltaTimeTravelSpec.scala:80)
	at com.databricks.sql.transaction.tahoe.commands.cdc.CDCReaderImpl.getVersionForCDC(CDCReader.scala:232)
	at com.databricks.sql.transaction.tahoe.commands.cdc.CDCReaderImpl.getCDCRelation(CDCReader.scala:289)
	at com.databricks.sql.transaction.tahoe.commands.cdc.CDCReaderImpl.getCDCRelation$(CDCReader.scala:276)
	at com.databricks.sql.transaction.tahoe.commands.cdc.CDCReader$.getCDCRelation(CDCReader.scala:68)
	at com.databricks.sql.

In [0]:
%sql
SELECT * FROM table_changes('demo','Timestamp1','Timestamp2' )

com.databricks.sql.transaction.tahoe.DeltaAnalysisException: [DELTA_TIMESTAMP_INVALID] The provided timestamp ('Timestamp') cannot be converted to a valid timestamp.
	at com.databricks.sql.transaction.tahoe.DeltaErrorsBase.timestampInvalid(DeltaErrors.scala:1470)
	at com.databricks.sql.transaction.tahoe.DeltaErrorsBase.timestampInvalid$(DeltaErrors.scala:1469)
	at com.databricks.sql.transaction.tahoe.DeltaErrors$.timestampInvalid(DeltaErrors.scala:3573)
	at com.databricks.sql.transaction.tahoe.DeltaTimeTravelSpec.getTimestamp(DeltaTimeTravelSpec.scala:80)
	at com.databricks.sql.transaction.tahoe.commands.cdc.CDCReaderImpl.getVersionForCDC(CDCReader.scala:232)
	at com.databricks.sql.transaction.tahoe.commands.cdc.CDCReaderImpl.getCDCRelation(CDCReader.scala:289)
	at com.databricks.sql.transaction.tahoe.commands.cdc.CDCReaderImpl.getCDCRelation$(CDCReader.scala:276)
	at com.databricks.sql.transaction.tahoe.commands.cdc.CDCReader$.getCDCRelation(CDCReader.scala:68)
	at com.databricks.sql.

In [0]:
%sql
SELECT * FROM table_changes('demo',5,7)

com.databricks.sql.transaction.tahoe.DeltaAnalysisException: [DELTA_TIMESTAMP_INVALID] The provided timestamp ('Timestamp') cannot be converted to a valid timestamp.
	at com.databricks.sql.transaction.tahoe.DeltaErrorsBase.timestampInvalid(DeltaErrors.scala:1470)
	at com.databricks.sql.transaction.tahoe.DeltaErrorsBase.timestampInvalid$(DeltaErrors.scala:1469)
	at com.databricks.sql.transaction.tahoe.DeltaErrors$.timestampInvalid(DeltaErrors.scala:3573)
	at com.databricks.sql.transaction.tahoe.DeltaTimeTravelSpec.getTimestamp(DeltaTimeTravelSpec.scala:80)
	at com.databricks.sql.transaction.tahoe.commands.cdc.CDCReaderImpl.getVersionForCDC(CDCReader.scala:232)
	at com.databricks.sql.transaction.tahoe.commands.cdc.CDCReaderImpl.getCDCRelation(CDCReader.scala:289)
	at com.databricks.sql.transaction.tahoe.commands.cdc.CDCReaderImpl.getCDCRelation$(CDCReader.scala:276)
	at com.databricks.sql.transaction.tahoe.commands.cdc.CDCReader$.getCDCRelation(CDCReader.scala:68)
	at com.databricks.sql.